<a href="https://colab.research.google.com/github/Alxmis/QR-decomposition/blob/Gram_Schmidt/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [2]:
import numpy as np
import cmath
import math

# Task for 6-7

In [3]:
def simple_qr(matrix: np.array) -> np.array:
    q, r = np.linalg.qr(matrix)
    for _ in range(1_000):
        matrix = r @ q
        q, r = np.linalg.qr(matrix)
    return matrix

def get_roots(matrix: np.array) -> np.array:
    n = matrix.shape[0]
    approximated = np.vstack([simple_qr(matrix), np.zeros(n)])
    determinated = approximated.round(5)

    roots = np.zeros(n)
    i = 0
    while i < n:
        root = approximated[i][i]
        detr = determinated[i+1][i]
        if detr == 0:
            roots[i] = root
        else:
            # select block
            p = 1
            for j in range(i+1, n):
                if determinated[j+1][j] == 0:
                    break
                p += 1
            block = approximated[i:i+p+1, i:i+p+1]
            # p == 1 -> our target
            if p == 1:
                D = (block[1][1] + block[0][0])**2 - 4*(block[0][0]*block[1][1]-block[0][1]*block[1][0])
                # convert type
                if D < 0:
                    roots = roots.astype(complex)
                    l1 = (block[1][1] + block[0][0] - cmath.sqrt(D))/2
                    l2 = (block[1][1] + block[0][0] + cmath.sqrt(D))/2
                else:
                    l1 = (block[1][1] + block[0][0] - math.sqrt(D))/2
                    l2 = (block[1][1] + block[0][0] + math.sqrt(D))/2
                roots[i + p - 1] = l1
                roots[i + p - 0] = l2
            # p != 1 -> not our target
            else:
                __roots = np.linalg.eigvals(block)
                for j in range(p+1):
                    roots[i + j] = __roots[j]
            print(block)
            i += p
        i += 1
    return roots


In [4]:
n = 3
omega = 50

matrix = np.random.randint(-omega, omega, (n, n))
print(f"Initial matrix:\n{matrix}")
approximated_matrix = simple_qr(matrix)
print(f"Approximated matrix:\n{approximated_matrix.round(5)}")
roots = get_roots(matrix)
print(f"Function`s roots:\n{roots}")
eigen = np.linalg.eigvals(matrix)
print(f"True eigenvalues:\n{eigen}")

Initial matrix:
[[  2 -13  38]
 [-28  40 -26]
 [-16 -27 -45]]
Approximated matrix:
[[ 56.13335  -8.41931  17.21504]
 [  0.      -27.16517  48.93792]
 [  0.       -3.73805 -31.96818]]
[[-27.16516969  48.93791817]
 [ -3.73804964 -31.9681846 ]]
Function`s roots:
[ 56.13335429 +0.j         -29.56667715-13.31033919j
 -29.56667715+13.31033919j]
True eigenvalues:
[-29.56667715+13.31033919j -29.56667715-13.31033919j
  56.13335429 +0.j        ]


# Task for 8

In [5]:
def hauseholder_qr(A):
    m, n = A.shape
    Q = np.eye(m)
    for i in range(n - (m == n)):
        H = np.eye(m)
        H[i:, i:] = make_householder(A[i:, i])
        Q = np.dot(Q, H)
        A = np.dot(H, A)
    return Q, A

def make_householder(a):
    v = a / (a[0] + np.copysign(np.linalg.norm(a), a[0]))
    v[0] = 1
    H = np.eye(a.shape[0])
    H -= (2 / np.dot(v, v)) * np.dot(v[:, None], v[None, :])
    return H

In [6]:
q_h, r_h = hauseholder_qr(matrix)
q, r = np.linalg.qr(matrix)

In [7]:
print(f'Householder\nQ:{q_h}\nR:{r_h}')
print(f'Numpy\nQ:{q}\nR:{r}')

Householder
Q:[[-0.06189845  0.25947574  0.96376393]
 [ 0.86657824 -0.4651077   0.18087833]
 [ 0.49518757  0.84637295 -0.19606659]]
R:[[-3.23109888e+01  2.20977452e+01 -4.71666159e+01]
 [ 5.93260382e-16 -4.48295623e+01 -1.61339042e+01]
 [ 4.74605187e-15  5.16632561e-15  4.07431894e+01]]
Numpy
Q:[[-0.06189845  0.25947574  0.96376393]
 [ 0.86657824 -0.4651077   0.18087833]
 [ 0.49518757  0.84637295 -0.19606659]]
R:[[-32.31098884  22.09774524 -47.1666159 ]
 [  0.         -44.82956229 -16.13390417]
 [  0.           0.          40.74318938]]


In [8]:
q_h.round(5) == q.round(5), r_h.round(5) == r.round(5)

(array([[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]]),
 array([[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]]))

# Task for 10 Gram_Schmidt

In [9]:
def gram_orthon(matrix):
    M = gram_orthog(matrix)
    n_cols = M.shape[1]
    for i in range(n_cols):
        M[:, i] = M[:, i] / np.linalg.norm(M[:, i])
    return M

def gram_orthog(matrix):
    n_rows, n_cols = matrix.shape
    M = np.zeros((n_rows, n_cols))
    for i in range(n_cols):
        m = matrix[:, i].astype('float64')
        for j in range(i):
            m -= np.dot(m, M[:, j]) / np.dot(M[:, j], M[:, j]) * M[:, j]
        M[:, i] = m
    return M

def gram_R(matrix, Q):
    n_rows, n_cols = matrix.shape
    R = np.zeros((n_rows, n_cols))
    for j in range(n_cols):
        for i in range(j + 1):
            R[i, j] = np.dot(Q[:, i], matrix[:, j])
    return R

def gram_qr(matrix):
  Q = gram_orthon(matrix)
  R = gram_R(matrix, Q)
  return Q, R

In [10]:
q_g, r_g = gram_qr(matrix)
q, r = np.linalg.qr(matrix)

In [11]:
print(f'Gram\nQ:{q_g}\nR:{r_g}\nQR:{q_g@r_g}')
print(f'Numpy\nQ:{q}\nR:{r}\nqr:{q@r}')

Gram
Q:[[ 0.06189845 -0.25947574  0.96376393]
 [-0.86657824  0.4651077   0.18087833]
 [-0.49518757 -0.84637295 -0.19606659]]
R:[[ 32.31098884 -22.09774524  47.1666159 ]
 [  0.          44.82956229  16.13390417]
 [  0.           0.          40.74318938]]
QR:[[  2. -13.  38.]
 [-28.  40. -26.]
 [-16. -27. -45.]]
Numpy
Q:[[-0.06189845  0.25947574  0.96376393]
 [ 0.86657824 -0.4651077   0.18087833]
 [ 0.49518757  0.84637295 -0.19606659]]
R:[[-32.31098884  22.09774524 -47.1666159 ]
 [  0.         -44.82956229 -16.13390417]
 [  0.           0.          40.74318938]]
qr:[[  2. -13.  38.]
 [-28.  40. -26.]
 [-16. -27. -45.]]


# Multihreading

In [12]:
import threading

In [64]:
cache = {}

def column(a: np.array, i: int) -> np.array:
    return np.array([x[i] for x in a])

def row(a: np.array, i: int) -> np.array:
    return a[i]

def dot(a: np.array, b: np.array, i: int) -> np.array:
    if isinstance(b, list):
        result = [0] * len(b)
        for j, data in enumerate(b):
            result[j] = dot(a, data, i)
        cache[i] = np.array(result)
    else:
        return np.sum(a * b.T)

def dot_product_threaded(a: np.array, b: np.array):
    assert a.shape[1] == b.shape[0]
    num_of_threads = a.shape[1]
    threads = []
    global cache
    cache = {}
    for t in range(num_of_threads):
        thread = threading.Thread(
            target=dot,
            args=(row(a, t), [column(b, i) for i in range(b.shape[1])], t)
        )
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    return np.array([cache[i] for i in range(num_of_threads)])

def qr_threaded(A: np.array, num_of_threads: int) -> np.array:
    n = A.shape[0]
    if n < num_of_threads:
        print('Number of threads is more than shape of matrix, changing...')
        num_of_threads = n
    blocks = []
    for i in range(num_of_threads):


In [82]:
n = 200
matrix_a = np.random.randint(-50, 50, (n, n))
matrix_b = np.random.randint(-50, 50, (n, n))
matrix_a, matrix_b

(array([[-33,  37, -11, ..., -16, -22,  -2],
        [-44, -41,   6, ...,  -6,  29, -34],
        [-38,  18,   7, ..., -36, -22,  -3],
        ...,
        [-42, -33,  18, ..., -16, -35,  37],
        [-37, -48,  47, ...,   8, -42,  34],
        [ 15,  -1,  46, ..., -31,  48, -17]]),
 array([[ 33, -13,  42, ..., -11, -43,  28],
        [ 11,  16,  11, ..., -39,  39,  -8],
        [-18, -13, -15, ...,  38, -27, -32],
        ...,
        [ 13,  25,   4, ..., -40,   9,   7],
        [ 31,  41,  44, ..., -20, -13, -12],
        [ 46, -26, -11, ...,  39,  30,  19]]))

In [83]:
dot_product_threaded(matrix_a, matrix_b)

array([[ -7376,  27329,   9940, ...,  11721,   8227, -11870],
       [-32191,   2780,  -9039, ...,  11207,  -6545,   2497],
       [  2007,  14984, -18070, ...,   2058, -14850,   4028],
       ...,
       [ -1249,   7623,   9347, ...,  18873,  -6625,   9008],
       [-12389,  -9929,   1656, ...,  10014,  -5580,   1051],
       [ 15462,  -4457,  -3405, ...,  -4567, -10396,  -3315]])

In [84]:
matrix_a @ matrix_b

array([[ -7376,  27329,   9940, ...,  11721,   8227, -11870],
       [-32191,   2780,  -9039, ...,  11207,  -6545,   2497],
       [  2007,  14984, -18070, ...,   2058, -14850,   4028],
       ...,
       [ -1249,   7623,   9347, ...,  18873,  -6625,   9008],
       [-12389,  -9929,   1656, ...,  10014,  -5580,   1051],
       [ 15462,  -4457,  -3405, ...,  -4567, -10396,  -3315]])

In [85]:
def hauseholder_qr_threaded(A):
    m, n = A.shape
    Q = np.eye(m)
    for i in range(n - (m == n)):
        H = np.eye(m)
        H[i:, i:] = make_householder(A[i:, i])
        Q = dot_product_threaded(Q, H)
        A = dot_product_threaded(H, A)
    return Q, A

In [86]:
n = 3
omega = 50

matrix = np.random.randint(-omega, omega, (n, n))

In [91]:
q_t, r_t = hauseholder_qr_threaded(matrix)
q, r = hauseholder_qr(matrix)

In [92]:
q_t, q

(array([[-0.23627012,  0.81669397,  0.52648588],
        [-0.76131483,  0.18109891, -0.62257763],
        [-0.60380142, -0.54791801,  0.57897297]]),
 array([[-0.23627012,  0.81669397,  0.52648588],
        [-0.76131483,  0.18109891, -0.62257763],
        [-0.60380142, -0.54791801,  0.57897297]]))